In [28]:
import torch
import torch.nn as nn
class GPP_pooling(nn.Module):
    def __init__(self,channel=64):
        super(GPP_pooling, self).__init__() 
        
        self.conv1=nn.Conv2d(channel,channel,1)
        self.conv2 =nn.Sequential(
            nn.Conv2d(channel,channel,1),
            nn.BatchNorm2d(num_features=channel),
            nn.ReLU(inplace=True)
        )
        
        self.channel = channel
        
        self.avepool = nn.AdaptiveAvgPool2d(1)
        self.avepool2 = nn.AdaptiveAvgPool2d((3,1))
        self.maxpool = nn.AdaptiveMaxPool2d(1)
    def forward(self,x):
        avg = self.avepool(x).squeeze(-1)
        avg2 = self.avepool2(x)
        max1 = self.maxpool(x)
        p_cot = self.conv1(avg2-max1.expand_as(avg2)).squeeze()
        p_cot_avg = torch.cat((avg,p_cot),dim=-1)
        gpp = self.conv2(p_cot_avg.unsqueeze(-1))
        print(p_cot.size(),p_cot_avg.size())
        return gpp.squeeze().mean(-1)
    
gpp = GPP_pooling()
a = torch.Tensor(64,64,256,128)
gpp = gpp(a)


torch.Size([64, 64, 3]) torch.Size([64, 64, 4])


In [32]:
class  GCPooling(nn.Module):
    def __init__(self,channel =64):
        super(GCPooling,self).__init__()
        self.conv1=nn.Conv2d(channel,channel,1)
        self.conv2 =nn.Sequential(
            nn.Conv2d(channel*2,channel,1),
            nn.BatchNorm2d(num_features=channel),
            nn.ReLU(inplace=True)
        )
        self.channel =channel
    def forward(self,max,avg):
        if (not self.training):
            return max,avg
        p_cont =self.conv1(avg-max)   ##(bs,channel,1)
        p_max = self.conv1(max)
        p_con_max = torch.cat((p_cont,p_max),dim=1)
        p_conmax = self.conv2(p_con_max)
        Gcp = p_conmax+max
        return Gcp.squeeze()
c= torch.Tensor(64,64,1,1)
d= torch.Tensor(64,64,1,1)
gcp= GCPooling()
f  = gcp(c,d)
print(f.size())

torch.Size([64, 64])
